### Optuna Tutorial 1 : 
https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/001_first.html#sphx-glr-tutorial-10-key-features-001-first-py


In [54]:
import optuna
def objective(trial):
    x = trial.suggest_float('x', -3,3) #add parameter to optimize 
    return (x-2)**2

In [55]:
##creating the study and actually running it 
"""
ftrial : single execution of the objective function

"""
study = optuna.create_study()
study.optimize(objective, n_trials=20)


[I 2022-12-15 07:00:05,858] A new study created in memory with name: no-name-f73a5d9c-0dbe-4a65-9ea4-a7d24e1e3f36
[I 2022-12-15 07:00:05,864] Trial 0 finished with value: 5.5384787646308 and parameters: {'x': -0.35339728151257965}. Best is trial 0 with value: 5.5384787646308.
[I 2022-12-15 07:00:05,866] Trial 1 finished with value: 4.444712417093395 and parameters: {'x': -0.1082486611150486}. Best is trial 1 with value: 4.444712417093395.
[I 2022-12-15 07:00:05,869] Trial 2 finished with value: 9.677057981205461 and parameters: {'x': -1.1107970009638142}. Best is trial 1 with value: 4.444712417093395.
[I 2022-12-15 07:00:05,870] Trial 3 finished with value: 1.9823178178378598 and parameters: {'x': 0.5920519122361578}. Best is trial 3 with value: 1.9823178178378598.
[I 2022-12-15 07:00:05,872] Trial 4 finished with value: 0.01086616640462463 and parameters: {'x': 2.1042409056207045}. Best is trial 4 with value: 0.01086616640462463.
[I 2022-12-15 07:00:05,873] Trial 5 finished with value

In [56]:
##looking at results 

print(study.best_params)

print(study.best_value)

print(study.best_trial)

print(study.trials_dataframe()) #get all trials as dataframe


{'x': 1.9385552073452568}
0.0037754625443843877
FrozenTrial(number=17, values=[0.0037754625443843877], datetime_start=datetime.datetime(2022, 12, 15, 7, 0, 5, 906325), datetime_complete=datetime.datetime(2022, 12, 15, 7, 0, 5, 907975), params={'x': 1.9385552073452568}, distributions={'x': UniformDistribution(high=3.0, low=-3.0)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=17, state=TrialState.COMPLETE, value=None)
    number      value             datetime_start          datetime_complete  \
0        0   5.538479 2022-12-15 07:00:05.862921 2022-12-15 07:00:05.864158   
1        1   4.444712 2022-12-15 07:00:05.866012 2022-12-15 07:00:05.866575   
2        2   9.677058 2022-12-15 07:00:05.868254 2022-12-15 07:00:05.868696   
3        3   1.982318 2022-12-15 07:00:05.870024 2022-12-15 07:00:05.870450   
4        4   0.010866 2022-12-15 07:00:05.871569 2022-12-15 07:00:05.872018   
5        5  15.519945 2022-12-15 07:00:05.873141 2022-12-15 07:00:05.873544   
6      

## Optuna Tutorial 2 : 
https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/002_configurations.html
https://github.com/optuna/optuna-examples/blob/main/pytorch/pytorch_simple.py
https://www.youtube.com/watch?v=P6NwZVl8ttc



### 2-0. 공통


In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms

In [2]:
#getting the dataloader dataset 
def get_mnist(batch_size):
    # Load FashionMNIST dataset.
    train_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(os.getcwd(), train=True, download=True, transform=transforms.ToTensor()),
        batch_size=batch_size,
        shuffle=True,
    )
    valid_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(os.getcwd(), train=False, transform=transforms.ToTensor()),
        batch_size=batch_size,
        shuffle=True,
    )

    return train_loader, valid_loader

### 2-1. Without Optuna

In [8]:
#hardcoding my hyperparameters in 
DEVICE = torch.device(0)
class my_model(nn.Module):
    def __init__(self):
        super(my_model, self).__init__()
        #parameters to be optimized later
        self.n_layers = 3 #will be optimized later
        
        layers = []
        in_features = 28*28  #input featues, for first layer is the img size flattened
        #use for loop to create model 
        for i in range(self.n_layers):
            #parameters to be optimized later
            out_features = 64 
            dropout_rate = 0.5
            
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate)) 
            
            in_features = out_features #so that it it is used proerly in next iteration
        
        layers.append(nn.Linear(in_features, 10)) #classificaiton layer
        layers.append(nn.LogSoftmax(dim = 1 )) #classification, normalize using softmax
        self.total = nn.Sequential(*layers) #create model 
            
    def forward(self, x):
        return self.total(x) 
    
    """
    parameters to be optimized if using optuna
    * self.n_layers
    * out_features (for each layer)
    * dropout_rate 
    """

    
###runing the actual modde
model = my_model().to(DEVICE)


##parameters to be optimized
lr = 1e-2
BATCHSIZE = 64
optimizer = optim.Adam(model.parameters(), lr = lr)


train_loader, valid_loader = get_mnist(batch_size = BATCHSIZE )

for epoch in range(10):
    model.train()
    
    #get training loss 
    for batch_idx, (data, target) in enumerate(train_loader):
        #limiting training dataset for faster epochs (일단은 무시)
        if batch_idx*BATCHSIZE > BATCHSIZE*30 :  #i.e. don't do many iterations (only a few per epoch)
            break
        
        #doing some stuff to data
        data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE) 
        
        #actual training
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()    
    print(f"EPOCH : {epoch} training loss, : {loss.item()}")
    #get valdiation loss
    model.eval()
    correct = 0 #count number of correct samples, to get accruacy
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(valid_loader):
            #limiting validation size 
            if batch_idx*BATCHSIZE > BATCHSIZE*10 : 
                break
            
            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE) 
            
            #actual validaiton 
            output = model(data)
            # Get the index of the max log-probability.
            pred = output.argmax(dim=1, keepdim=True)
            loss = F.nll_loss(output, target)
            
            
            correct += pred.eq(target.view_as(pred)).sum().item()
        
    accuracy = correct / min(len(valid_loader.dataset), BATCHSIZE*10) 
        
    print(f"EPOCH : {epoch} validation loss, accuracy : {loss} / {accuracy}") 

EPOCH : 0 training loss, : 1.44558846950531
EPOCH : 0 validation loss, accuracy : 1.2670201063156128 / 0.5609375
EPOCH : 1 training loss, : 1.2591526508331299
EPOCH : 1 validation loss, accuracy : 1.0117079019546509 / 0.6578125
EPOCH : 2 training loss, : 1.1057322025299072
EPOCH : 2 validation loss, accuracy : 0.986359715461731 / 0.6828125
EPOCH : 3 training loss, : 1.3560160398483276
EPOCH : 3 validation loss, accuracy : 0.9358760118484497 / 0.703125
EPOCH : 4 training loss, : 1.24252450466156
EPOCH : 4 validation loss, accuracy : 0.7710863947868347 / 0.7625
EPOCH : 5 training loss, : 1.2464555501937866
EPOCH : 5 validation loss, accuracy : 0.7817651033401489 / 0.8078125
EPOCH : 6 training loss, : 1.2980538606643677
EPOCH : 6 validation loss, accuracy : 0.8685086965560913 / 0.7015625
EPOCH : 7 training loss, : 1.2039804458618164
EPOCH : 7 validation loss, accuracy : 1.006385087966919 / 0.715625
EPOCH : 8 training loss, : 1.152542233467102
EPOCH : 8 validation loss, accuracy : 0.810612

### 2-2. With Optuna

In [9]:
import optuna
#hardcoding my hyperparameters in 
DEVICE = torch.device(0)

##model takes in trial as input 
class my_model(nn.Module):
    def __init__(self, trial):
        super(my_model, self).__init__()
        #parameters to be optimized later
        self.n_layers = trial.suggest_int(name = "n_layers", low =1, high = 3) 
        
        layers = []
        in_features = 28*28  #input featues, for first layer is the img size flattened
        #use for loop to create model 
        for i in range(self.n_layers):
            #parameters to be optimized later
            #out_features = 64 
            out_features = trial.suggest_int(f"n_units_l{i}", 4, 64)
            #dropout_rate = 0.5
            dropout_rate = trial.suggest_float(f"dropout_l{i}", 0.00001, 0.5)
            
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate)) 
            
            in_features = out_features #so that it it is used proerly in next iteration
        
        layers.append(nn.Linear(in_features, 10)) #classificaiton layer
        layers.append(nn.LogSoftmax(dim = 1 )) #classification, normalize using softmax
        self.total = nn.Sequential(*layers) #create model 
            
    def forward(self, x):
        return self.total(x) 
    
    """
    parameters to be optimized if using optuna
    * self.n_layers
    * out_features (for each layer)
    * dropout_rate 
    """                     
    

def objective(trial):
    ###runing the actual modde
    #model = my_model().to(DEVICE)
    model = my_model(trial).to(DEVICE) #must put trial in 
    
    
    ##parameters to be optimized
    lr = trial.suggest_float("learning_rate", 1e-3, 1e-1, log = True)
    BATCHSIZE = trial.suggest_int("batch_size", 16, 128, log = True)
    optimizer_choice = trial.suggest_categorical('optimizer', ["Adam", "RMSprop", "SGD"])

    
    #using the trial suggest things and use them to define our wanted things
    optimizer = getattr(optim, optimizer_choice)(model.parameters(), lr = lr) 
    train_loader, valid_loader = get_mnist(batch_size = BATCHSIZE )
    
    for epoch in range(10):
        model.train()
        
        #get training loss 
        for batch_idx, (data, target) in enumerate(train_loader):
            #limiting training dataset for faster epochs (일단은 무시)
            if batch_idx*BATCHSIZE > BATCHSIZE*30 :  #i.e. don't do many iterations (only a few per epoch)
                break
            
            #doing some stuff to data
            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE) 
            
            #actual training
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()    
        print(f"EPOCH : {epoch} training loss, : {loss.item()}")
        #get valdiation loss
        model.eval()
        correct = 0 #count number of correct samples, to get accruacy
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                #limiting validation size 
                if batch_idx*BATCHSIZE > BATCHSIZE*10 : 
                    break
                
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE) 
                
                #actual validaiton 
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                loss = F.nll_loss(output, target)
                
                
                correct += pred.eq(target.view_as(pred)).sum().item()
            
        accuracy = correct / min(len(valid_loader.dataset), BATCHSIZE*10) 
        
        trial.report(accuracy, epoch)
            
        print(f"EPOCH : {epoch} validation loss, accuracy : {loss} / {accuracy}") 
        
    return accuracy #the final thing we want to maximize 
        

In [10]:
##actual pruning 
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials= 10, timeout= 600)


print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))


print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items(): #get dict values 
    print("    {}: {}".format(key, value))






[I 2022-12-15 08:13:02,080] A new study created in memory with name: no-name-bc0e523b-03cd-4371-bee9-5c06ad655840


EPOCH : 0 training loss, : 2.3051068782806396
EPOCH : 0 validation loss, accuracy : 2.2711310386657715 / 0.14347826086956522
EPOCH : 1 training loss, : 2.30908203125
EPOCH : 1 validation loss, accuracy : 2.313206911087036 / 0.15217391304347827
EPOCH : 2 training loss, : 2.326399803161621
EPOCH : 2 validation loss, accuracy : 2.307403564453125 / 0.11304347826086956
EPOCH : 3 training loss, : 2.249756097793579
EPOCH : 3 validation loss, accuracy : 2.3247530460357666 / 0.11304347826086956
EPOCH : 4 training loss, : 2.3093459606170654
EPOCH : 4 validation loss, accuracy : 2.297776699066162 / 0.10869565217391304
EPOCH : 5 training loss, : 2.312589645385742
EPOCH : 5 validation loss, accuracy : 2.3328182697296143 / 0.1
EPOCH : 6 training loss, : 2.336906909942627
EPOCH : 6 validation loss, accuracy : 2.3381197452545166 / 0.11304347826086956
EPOCH : 7 training loss, : 2.308753252029419
EPOCH : 7 validation loss, accuracy : 2.3415067195892334 / 0.1


[I 2022-12-15 08:13:04,009] Trial 0 finished with value: 0.10434782608695652 and parameters: {'n_layers': 1, 'n_units_l0': 19, 'dropout_l0': 0.17855996046819958, 'learning_rate': 0.08900813792059299, 'batch_size': 23, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.10434782608695652.


EPOCH : 8 training loss, : 2.2986466884613037
EPOCH : 8 validation loss, accuracy : 2.275066614151001 / 0.1
EPOCH : 9 training loss, : 2.3049588203430176
EPOCH : 9 validation loss, accuracy : 2.2854137420654297 / 0.10434782608695652
EPOCH : 0 training loss, : 1.6498234272003174
EPOCH : 0 validation loss, accuracy : 1.4582188129425049 / 0.37894736842105264
EPOCH : 1 training loss, : 1.5474704504013062
EPOCH : 1 validation loss, accuracy : 1.3117706775665283 / 0.55
EPOCH : 2 training loss, : 2.0586979389190674
EPOCH : 2 validation loss, accuracy : 1.3011032342910767 / 0.5210526315789473
EPOCH : 3 training loss, : 1.7187128067016602
EPOCH : 3 validation loss, accuracy : 1.3362942934036255 / 0.5289473684210526
EPOCH : 4 training loss, : 2.475966691970825
EPOCH : 4 validation loss, accuracy : 1.1579374074935913 / 0.5605263157894737
EPOCH : 5 training loss, : 1.5992028713226318
EPOCH : 5 validation loss, accuracy : 1.3074030876159668 / 0.5
EPOCH : 6 training loss, : 1.579829454421997
EPOCH :

[I 2022-12-15 08:13:06,695] Trial 1 finished with value: 0.5447368421052632 and parameters: {'n_layers': 2, 'n_units_l0': 25, 'dropout_l0': 0.2909773299760975, 'n_units_l1': 33, 'dropout_l1': 0.27700146118102575, 'learning_rate': 0.039969510756996396, 'batch_size': 38, 'optimizer': 'Adam'}. Best is trial 1 with value: 0.5447368421052632.


...

EPOCH : 9 training loss, : 2.2960150241851807
EPOCH : 9 validation loss, accuracy : 2.3062727451324463 / 0.09195402298850575
EPOCH : 0 training loss, : 0.7982692122459412
EPOCH : 0 validation loss, accuracy : 0.7190901637077332 / 0.8209876543209876
EPOCH : 1 training loss, : 0.5741562843322754
EPOCH : 1 validation loss, accuracy : 0.6055198311805725 / 0.8530864197530864
EPOCH : 2 training loss, : 0.48095476627349854
EPOCH : 2 validation loss, accuracy : 0.6871677041053772 / 0.8827160493827161
EPOCH : 3 training loss, : 0.49216723442077637
EPOCH : 3 validation loss, accuracy : 0.6058894991874695 / 0.854320987654321
EPOCH : 4 training loss, : 0.6380926966667175
EPOCH : 4 validation loss, accuracy : 0.6824563145637512 / 0.8777777777777778
EPOCH : 5 training loss, : 0.49605512619018555
EPOCH : 5 validation loss, accuracy : 0.563435435295105 / 0.8839506172839506
EPOCH : 6 training loss, : 0.6299576759338379
EPOCH : 6 validation loss, accuracy : 0.5444654822349548 / 0.8765432098765432
EPOCH 

[I 2022-12-15 08:13:31,651] Trial 8 finished with value: 0.9098765432098765 and parameters: {'n_layers': 1, 'n_units_l0': 33, 'dropout_l0': 0.051787921861834556, 'learning_rate': 0.007035245494009382, 'batch_size': 81, 'optimizer': 'Adam'}. Best is trial 3 with value: 0.910126582278481.


EPOCH : 9 training loss, : 0.32724499702453613
EPOCH : 9 validation loss, accuracy : 0.2949514091014862 / 0.9098765432098765
EPOCH : 0 training loss, : 1.2016900777816772
EPOCH : 0 validation loss, accuracy : 1.1662019491195679 / 0.6660377358490566
EPOCH : 1 training loss, : 1.107042670249939
EPOCH : 1 validation loss, accuracy : 0.950439453125 / 0.7622641509433963
EPOCH : 2 training loss, : 1.0517178773880005
EPOCH : 2 validation loss, accuracy : 0.7939538955688477 / 0.7528301886792453
EPOCH : 3 training loss, : 0.8979871273040771
EPOCH : 3 validation loss, accuracy : 0.6665497422218323 / 0.7735849056603774
EPOCH : 4 training loss, : 0.8266420364379883
EPOCH : 4 validation loss, accuracy : 0.8141704797744751 / 0.8018867924528302
EPOCH : 5 training loss, : 0.9853619933128357
EPOCH : 5 validation loss, accuracy : 0.6295116543769836 / 0.7754716981132076
EPOCH : 6 training loss, : 0.9803934693336487
EPOCH : 6 validation loss, accuracy : 0.6702020764350891 / 0.8226415094339623
EPOCH : 7 tr

[I 2022-12-15 08:13:34,799] Trial 9 finished with value: 0.8415094339622642 and parameters: {'n_layers': 1, 'n_units_l0': 36, 'dropout_l0': 0.3579131750257493, 'learning_rate': 0.0012738904784910588, 'batch_size': 53, 'optimizer': 'RMSprop'}. Best is trial 3 with value: 0.910126582278481.


EPOCH : 9 training loss, : 0.9847554564476013
EPOCH : 9 validation loss, accuracy : 0.5504941344261169 / 0.8415094339622642
Study statistics: 
  Number of finished trials:  10
Best trial:
  Value:  0.910126582278481
  Params: 
    n_layers: 2
    n_units_l0: 47
    dropout_l0: 0.053321589361190976
    n_units_l1: 42
    dropout_l1: 0.12176002475279113
    learning_rate: 0.0039108232529245
    batch_size: 79
    optimizer: Adam


In [6]:
optuna.trial.Trial.suggest_float()
optuna.trial.Trial.suggest_int()

TypeError: suggest_float() missing 4 required positional arguments: 'self', 'name', 'low', and 'high'

### Optuna CV
https://stackoverflow.com/questions/63224426/how-can-i-cross-validate-by-pytorch-and-optuna
